In [ ]:
from evdev import InputDevice, ecodes
device = InputDevice('/dev/input/event0')   # Events for Xbox One S Wireless Controller

In [ ]:
import time, os, ssl, time, queue, socket, struct, random, threading
q = queue.Queue() 
def heartbeatd(): 
    while True:
        q.join()
        print('heartbeat')
        q.put(struct.pack('<hi', -1, 0)) # an invalid acton to keep the connection alive, would be ignored by rover but prevent a timeout for the control link
        time.sleep(1)
threading.Thread(target=heartbeatd).start() 

In [ ]:
def controllerd(): 
    for event in device.read_loop(): 
        if event.type in [ecodes.EV_ABS, ecodes.EV_KEY]: 
            q.join()
            print('xbox')
            q.put(struct.pack('<hi', event.code, event.value)) # put it into a queue
threading.Thread(target=controllerd).start() 

In [ ]:
# Establish connection with server

server_ip = 'guoxiaokang.net'
ctrl_port = 54322 
server_cert = 'secret/server.crt'
client_cert = 'secret/driver.crt'
client_key  = 'secret/driver.key'

context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert)
context.load_cert_chain(certfile=client_cert, keyfile=client_key) 

while True: 
    try: 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sc = context.wrap_socket(s, server_side=False, server_hostname=server_ip) 
        sc.settimeout(30)  
        print('Try to connect the server')
        sc.connect((server_ip, ctrl_port))  
        print('Succeed a connection') 
        while True:
            item = q.get()
            print(item)
            sc.send(item) # Send code and value to server 
            q.task_done()
    except socket.timeout as e: 
        sc.shutdown(socket.SHUT_RDWR)
        sc.close()  
    except BrokenPipeError:
        pass
    except ConnectionRefusedError:
        print('Failed to connect the server and retry the connection in 5 seconds')
        time.sleep(5)
    except ConnectionResetError:
        print('Server is restarting and connection will be retried in 5 seconds')
        time.sleep(5)
    finally:
        q.task_done()